In [16]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse

import locale
import time
from datetime import datetime
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

from tqdm import tqdm

In [17]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf.loc[:,'IP Address'].values[0]
}

### object detail
id: int  
screen_name: @usuario  
tweet_count: int  
description: descripcion usuario 
location: ubicacion en desc  
url: url en desc  
listed_count: listas de las cuales forma parte  
created_at: cuenta creada el  
crawled_at: cuando se escrapeo  
favs_count: int  
followers_count: int  
following_count: int
verified: true/false

In [20]:
#NO DESCOMENTAR!
# userDf = pd.DataFrame(columns=[
#     'id',
#     'screen_name',
#     'complete_name',
#     'tweet_count',
#     'description',
#     'location',
#     'birthdate',
#     'url',
#     'listed_count',
#     'created_at',
#     'crawled_at',
#     'favs_count',
#     'followers_count',
#     'following_count',
#     'verified'
# ])

userDf = pd.read_pickle('../datasets/UsersDataset.pkl')
toScrap = pd.read_pickle('../datasets/targetUsersDataset.pkl')

#saco los que ya escrapie
display(userDf.head(2))
print('Usuarios ya existentes', len(userDf))
print('\n\n')
display(toScrap.head(2))
print('Targets totales', len(toScrap))
toScrap.index = toScrap.index.map(int)
toScrap = toScrap.loc[toScrap.index.difference(userDf.index).values]
print('Target restantes', len(toScrap))

,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
index,,,,,,,,,,,,,,,
2300560809,2300560809,NMoraniB,Nicole Morani Brown,593,Gerencia General de @PuertoBsAs Ministerio de ...,"Buenos Aires, Argentina",False,False,0,2014-01-01 00:00:00,2019-06-29 23:04:21.059135,2377,1630,1510,False
1536923558,1536923558,FabianPereyra87,Fabian Pereyra,2532,,False,False,False,0,2013-06-01 00:00:00,2019-06-29 23:04:21.289518,4307,53,284,False


Usuarios ya existentes 8719





,username
235491971,gsrcaballero
497428913,juannfalopa


Targets totales 17205
Target restantes 8487


In [21]:
def scrap_users(toScrap):
    for username in tqdm(toScrap):
        try:
            pageResponse = requests.get('https://twitter.com/'+username, headers=headersChrome, proxies=proxies)
            pageParsed = BeautifulSoup(pageResponse.text, 'html.parser')
            profileNavItems = {
                'tweets': 0,
                'following': 0,
                'followers': 0,
                'favorites': 0,
                'moments': 0,
                'lists': 0
            }
            for itemName in profileNavItems.keys():
                liSearch = pageParsed.findAll(attrs={'class':'ProfileNav-item--'+itemName})
                if(len(liSearch)>0):
                    liValue = liSearch[0].findAll(attrs={'class':'ProfileNav-value'})[0]
                    if( 'data-count' in  liValue.attrs.keys()):
                        profileNavItems[itemName] = liValue.attrs['data-count']
                    else:
                        profileNavItems[itemName] = liValue.text
            completeName = pageParsed.findAll('h1', attrs={'class':'ProfileHeaderCard-name'})[0].findAll('a')[0].text
            completeName = completeName.replace(u'\xa0', u' ')
            completeName = completeName.replace(u'\n', ' ')
            bio = pageParsed.findAll(attrs={'class':'ProfileHeaderCard-bio'})[0].text
            bio = bio.replace(u'\xa0', u' ')
            bio = bio.replace(u'\n', ' ')
            isVerified = False
            if(len(pageParsed.findAll(attrs={'class':'ProfileHeaderCard-name'})[0].findAll(attrs={'class':'Icon--verified'}))>0):
                isVerified = True
            userId = int(pageParsed.findAll(attrs={'class':'ProfileNav'})[0].attrs['data-user-id'])
            profileHeaderItems = {
                'location': False,
                'url': False,
                'joinDate': False,
                'birthdate': False
            }
            for itemName in profileHeaderItems.keys():
                liSearch = pageParsed.findAll(attrs={'class':'ProfileHeaderCard-'+itemName})
                if(len(liSearch)>0):
                    if('u-hidden' not in liSearch[0].attrs['class']):
                        liValue = liSearch[0].findAll(attrs={'class':'ProfileHeaderCard-'+itemName+'Text'})[0].text
                        liValue = re.sub(u'\n', ' ', liValue)
                        liValue = liValue.strip(' ')
                        liValue = liValue.replace(u'\xa0', u' ')
                        if(itemName == 'joinDate'):
                            liValue = re.sub(r'Se unió en ', '', liValue)
                            liValue = re.sub(r' de ', ' ', liValue)
                            liValue = datetime.strptime(liValue, '%B %Y')
                            liValue = pd.Timestamp(ts_input=liValue)
                        if(itemName == 'birthdate'):
                            liValue = re.sub(r'Fecha de nacimiento ', '', liValue)
                        profileHeaderItems[itemName] = liValue
            userData = {
                'id': userId,
                'screen_name': username,
                'complete_name': completeName,
                'description': bio,
                'verified': isVerified,
                'crawled_at': pd.Timestamp.now(),
                'location': profileHeaderItems['location'],
                'url': profileHeaderItems['url'],
                'created_at': profileHeaderItems['joinDate'],
                'birthdate': profileHeaderItems['birthdate'],
                'tweet_count': profileNavItems['tweets'],
                'listed_count': profileNavItems['lists'],
                'favs_count': profileNavItems['favorites'],
                'followers_count': profileNavItems['followers'],
                'following_count': profileNavItems['following']

            }
            userDf.loc[userData['id']] = userData
        except:
            print('ups')

In [22]:
# Funcion para Fraccionar el universo a scrapear
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

In [23]:
# Función para paralelizar
fracciones = 10
cuenta_errores = 0
from joblib import Parallel, delayed

def parascrap(arg):
    cuenta_errores = 0
    try:
        scrap_users(arg)
        return result
    except:
        print('Error')
        cuenta_errores += 1

result = Parallel(n_jobs=fracciones, backend="threading")(map(delayed(parascrap),chunkIt(toScrap.username.values,fracciones)))










  0%|                                                                                          | 0/849 [00:00<?, ?it/s]








  0%|                                                                                          | 0/849 [00:00<?, ?it/s]









  0%|                                                                                          | 0/848 [00:00<?, ?it/s]










  0%|                                                                                          | 0/849 [00:00<?, ?it/s]











  0%|                                                                                          | 0/848 [00:00<?, ?it/s]












  0%|                                                                                          | 0/849 [00:00<?, ?it/s]













  0%|                                                                                          | 0/848 [00:00<?, ?it/s]














  0%|                                                                      

ups

  0%|                                                                                | 1/848 [00:08<1:51:37,  7.91s/it]













  0%|                                                                                | 1/848 [00:08<1:58:15,  8.38s/it]















  0%|                                                                                | 1/849 [00:08<1:58:05,  8.36s/it]













  0%|                                                                                | 1/848 [00:08<1:58:52,  8.42s/it]














  0%|                                                                                | 1/849 [00:08<1:58:59,  8.42s/it]








  0%|                                                                                | 1/849 [00:08<2:01:20,  8.59s/it]
















  0%|                                                                                | 1/849 [00:08<2:01:13,  8.58s/it]







  0%|▏                                                                               | 2/849 [00:13<1:33:11,  6.60s/it]












  0%|▏                                                         

  2%|█▏                                                                             | 13/848 [01:41<1:49:11,  7.85s/it]













  2%|█▎                                                                             | 14/848 [01:42<1:46:46,  7.68s/it]
















  2%|█▏                                                                             | 13/849 [01:42<1:44:32,  7.50s/it]












  2%|█▏                                                                             | 13/849 [01:42<1:45:01,  7.54s/it]








  2%|█▎                                                                             | 14/849 [01:43<1:46:45,  7.67s/it]














  2%|█▎                                                                             | 14/849 [01:44<1:37:40,  7.02s/it]










  2%|█▎                                                                             | 14/849 [01:45<1:46:47,  7.67s/it]







  2%|█▍                                                                         

...

ups


 98%|████████████████████████████████████████████████████████████████████████████▍ | 832/849 [1:35:53<01:58,  6.94s/it]












 94%|█████████████████████████████████████████████████████████████████████████▌    | 801/849 [1:35:53<05:24,  6.76s/it]











 95%|██████████████████████████████████████████████████████████████████████████▎   | 808/848 [1:35:54<04:40,  7.01s/it]















100%|█████████████████████████████████████████████████████████████████████████████▋| 845/849 [1:35:54<00:23,  5.78s/it]








 98%|████████████████████████████████████████████████████████████████████████████▎ | 831/849 [1:35:54<02:02,  6.80s/it]









 96%|██████████████████████████████████████████████████████████████████████████▊   | 814/848 [1:35:54<03:14,  5.73s/it]













 96%|██████████████████████████████████████████████████████████████████████████▊   | 814/848 [1:35:56<03:38,  6.43s/it]







 98%|████████████████████████████████████████████████████████████████████████████▌ | 

 98%|████████████████████████████████████████████████████████████████████████████▎ | 830/848 [1:37:38<01:04,  3.61s/it]










 97%|███████████████████████████████████████████████████████████████████████████▍  | 821/849 [1:37:39<02:06,  4.51s/it]











 98%|████████████████████████████████████████████████████████████████████████████▋ | 834/848 [1:37:39<00:55,  3.97s/it]







 98%|████████████████████████████████████████████████████████████████████████████▊ | 835/848 [1:37:39<00:46,  3.56s/it]









 98%|████████████████████████████████████████████████████████████████████████████▍ | 831/848 [1:37:42<01:06,  3.89s/it]










 97%|███████████████████████████████████████████████████████████████████████████▌  | 822/849 [1:37:43<01:55,  4.27s/it]







 99%|████████████████████████████████████████████████████████████████████████████▉ | 836/848 [1:37:43<00:43,  3.60s/it]











 98%|████████████████████████████████████████████████████████████████████████████▊ | 835/848 [1

KeyboardInterrupt: 











 98%|████████████████████████████████████████████████████████████████████████████▋ | 835/849 [1:38:29<00:51,  3.66s/it]








100%|█████████████████████████████████████████████████████████████████████████████▋| 844/848 [1:38:30<00:15,  3.90s/it]










100%|█████████████████████████████████████████████████████████████████████████████▊| 846/848 [1:38:30<00:08,  4.16s/it]









 98%|████████████████████████████████████████████████████████████████████████████▊ | 836/849 [1:38:32<00:43,  3.33s/it]








100%|█████████████████████████████████████████████████████████████████████████████▋| 845/848 [1:38:32<00:10,  3.56s/it]










100%|█████████████████████████████████████████████████████████████████████████████▉| 847/848 [1:38:32<00:03,  3.73s/it]









 99%|████████████████████████████████████████████████████████████████████████████▉ | 837/849 [1:38:35<00:38,  3.21s/it]








100%|█████████████████████████████████████████████████████████████████████████████▊| 846

In [24]:
print('El scrapeo fallo para ',cuenta_errores, ' usuarios')
userDf.info()

El scrapeo fallo para  0  usuarios
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15482 entries, 2300560809 to 1493451997
Data columns (total 15 columns):
id                 15482 non-null object
screen_name        15482 non-null object
complete_name      15482 non-null object
tweet_count        15482 non-null object
description        15482 non-null object
location           15482 non-null object
birthdate          15482 non-null object
url                15482 non-null object
listed_count       15482 non-null object
created_at         15482 non-null object
crawled_at         15482 non-null object
favs_count         15482 non-null object
followers_count    15482 non-null object
following_count    15482 non-null object
verified           15482 non-null object
dtypes: object(15)
memory usage: 1.9+ MB


In [25]:
# Descargar a un archivo
userDf.to_pickle('../datasets/UsersDataset.pkl')

upsupsups














 61%|███████████████████████████████████████████▋                            | 515/849 [3:11:41<213:36:47, 2302.42s/it]







 61%|███████████████████████████████████████████▉                            | 518/849 [3:11:41<211:36:13, 2301.43s/it]

ups

 62%|████████████████████████████████████████████▌                           | 525/849 [3:11:41<207:01:36, 2300.30s/it]

ups
ups
ups
upsups

ups
ups
upsups

ups
ups
upsups
ups

upsups
ups

ups
ups
ups
upsups

ups
upsups

ups
upsups

ups
upsups

ups
upsups

upsups
ups


...

100%|█████████████████████████████████████████████████████████████████████████████▋| 845/849 [3:11:51<00:00, 38.63it/s]








100%|█████████████████████████████████████████████████████████████████████████████▋| 845/849 [3:11:51<00:00, 32.64it/s]

ups

100%|██████████████████████████████████████████████████████████████████████████████| 849/849 [3:11:51<00:00, 13.56s/it]

ups
upsups

ups
ups
upsups












100%|██████████████████████████████████████████████████████████████████████████████| 849/849 [3:11:51<00:00, 31.60it/s]







100%|██████████████████████████████████████████████████████████████████████████████| 849/849 [3:11:51<00:00, 35.31it/s]